# Ejercicios RDD de pares
1. Inicia las variables SparkSession y SparkContext (0)

In [1]:

# Inicializamos SparkSession y SparkContext
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("02-rddej2") \
    .getOrCreate()

sc = spark.sparkContext
print(spark.version)  # Verifica la versión de Spark


:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/hadoop/.ivy2/cache
The jars for the packages stored in: /home/hadoop/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.kafka#kafka-clients added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-4b59fdc3-3f3b-4053-96b8-156a18317b9e;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.1.0 in central
	found io.delta#delta-storage;3.1.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.5.7 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.5.7 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found org.xerial.snappy#snappy-java;1.1.10.5 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.fi

3.5.7


----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 50786)
Traceback (most recent call last):
  File "/usr/lib/python3.8/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python3.8/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python3.8/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python3.8/socketserver.py", line 747, in __init__
    self.handle()
  File "/usr/local/lib/python3.8/dist-packages/pyspark/accumulators.py", line 295, in handle
    poll(accum_updates)
  File "/usr/local/lib/python3.8/dist-packages/pyspark/accumulators.py", line 267, in poll
    if self.rfile in r and func():
  File "/usr/local/lib/python3.8/dist-packages/pyspark/accumulators.py", line 271, in accum_updates
    num_updates = read_int(self.rf

2. Crea un rdd de pares da forma (palabra, 1) a partir do ficheiro /data/el_quijote.txt (0,25)

In [3]:
# 2) RDD de pares (palabra, 1) a partir de data/el_quijote.txt

rdd_palabras_1 = (
    sc.textFile("/data/el_quijote.txt")
      .flatMap(lambda liña: liña.split())
      .map(lambda palabra: (palabra, 1))
)

rdd_palabras_1.take(10)


[('DON', 1),
 ('QUIJOTE', 1),
 ('DE', 1),
 ('LA', 1),
 ('MANCHA', 1),
 ('Miguel', 1),
 ('de', 1),
 ('Cervantes', 1),
 ('Saavedra', 1),
 ('PRIMERA', 1)]

3. A partir do RDD anterior, conta o número de aparicións de cada palabra (0,5)


In [4]:
# 3) Contar o número de aparicións de cada palabra a partir de (palabra, 1)
rdd_conta_palabras = rdd_palabras_1.reduceByKey(lambda a, b: a + b)

rdd_conta_palabras.take(10)


[('DE', 17),
 ('LA', 13),
 ('Miguel', 3),
 ('PRIMERA', 1),
 ('CAPÍTULO', 1),
 ('1:', 1),
 ('condición', 23),
 ('y', 8042),
 ('del', 1113),
 ('hidalgo', 16)]

4. Repite os exercicios 2 e 3 de xeito que non se teñan en conta os signos de puntuación e non se distingan maiúsculas de minúsculas (mapea todo a minúsculas) (0,25)


In [6]:
import re

# 4) Repetir 2 e 3 ignorando puntuación e sen distinguir maiúsculas/minúsculas
rdd_palabras_1_limpo = (
    sc.textFile("/data/el_quijote.txt")
      .flatMap(lambda liña: re.findall(r"[a-záéíóúüñ]+", liña.lower()))
      .map(lambda palabra: (palabra, 1))
)

rdd_conta_palabras_limpo = rdd_palabras_1_limpo.reduceByKey(lambda a, b: a + b)

rdd_conta_palabras_limpo.take(10)


[('no', 3159),
 ('s', 1571),
 ('mis', 171),
 ('alabanzas', 13),
 ('don', 1088),
 ('porque', 772),
 ('nero', 30),
 ('adulacio', 2),
 ('y', 8690),
 ('lo', 1945)]

5. Agregacións.
   - a) `keyBy`: rear rdd_inicial onde a clave sexa a primeira letra de cada palabra de rdd_texto (0,5).
   - b)Partindo de `rdd_inicial` (1):
     - Converter o valor (palabra) a **maiúsculas** usando `mapValues`.
     - Crear outro RDD onde o valor sexa a **lonxitude** da palabra, mantendo a mesma clave.
   - c) `keys`: Obter as claves distintas de `rdd_inicial`(0,5)
   - d) `values`: Obter cantos valores distintos hai en `rdd_inicial` (0,5).
   - e) `sample_by_key` (1):
     - En `rdd_inicial`, tomar unha mostra do **80%** das palabras que comezan por `'d'` e **20%** das que comezan por `'l'`.
     - Evitar `KeyError` asegurando que o diccionario `fractions` inclúe **todas as claves** (as non usadas deben ter 0.0).
   - f) `countByKey`: contar elementos por clave de `rdd_inicial` (0,5).
   - g) `aggregate`: sobre `rdd_texto` (1):
     -   Calcula nunha soa pasada `(n_palabras, suma_caracteres)`.
     -   Obtén a `lonxitude_media`.

In [7]:
# Dataset base para todo o laboratorio
# (executar esta celda antes de comezar cos exercicios)

# 1) Texto (para keyBy / mapValues / reduceByKey...)
texto = """En un lugar de la Mancha de cuyo nombre no quiero acordarme
no ha mucho tiempo que vivía un hidalgo de los de lanza en astillero
adarga antigua rocín flaco y galgo corredor"""
rdd_texto = sc.parallelize(texto.lower().split())



In [8]:
# a
rdd_inicial = rdd_texto.keyBy(lambda w: w[0])
rdd_inicial.take(10)

[('e', 'en'),
 ('u', 'un'),
 ('l', 'lugar'),
 ('d', 'de'),
 ('l', 'la'),
 ('m', 'mancha'),
 ('d', 'de'),
 ('c', 'cuyo'),
 ('n', 'nombre'),
 ('n', 'no')]

In [9]:
# b
rdd_inicial_upper = rdd_inicial.mapValues(lambda w: w.upper())
rdd_inicial_len = rdd_inicial.mapValues(lambda w: len(w))
rdd_inicial_upper.take(5), rdd_inicial_len.take(5)

([('e', 'EN'), ('u', 'UN'), ('l', 'LUGAR'), ('d', 'DE'), ('l', 'LA')],
 [('e', 2), ('u', 2), ('l', 5), ('d', 2), ('l', 2)])

In [10]:
# c
claves_distintas = rdd_inicial.keys().distinct().collect()
claves_distintas

['l',
 'd',
 'c',
 'h',
 'r',
 'y',
 'g',
 'q',
 'v',
 'u',
 'e',
 'a',
 'f',
 'm',
 'n',
 't']

In [11]:
# d
n_valores_distintos = rdd_inicial.values().distinct().count()
n_valores_distintos

27

In [12]:
# e
keys = rdd_inicial.keys().distinct().collect()
fractions = {k: 0.0 for k in keys}
fractions['d'] = 0.8
fractions['l'] = 0.2
mostra = rdd_inicial.sampleByKey(False, fractions, seed=42)
mostra.take(20)

[('d', 'de'), ('l', 'la'), ('d', 'de'), ('d', 'de'), ('l', 'lanza')]

In [13]:
# f
conta_por_clave = rdd_inicial.countByKey()
conta_por_clave

defaultdict(int,
            {'e': 2,
             'u': 2,
             'l': 4,
             'd': 4,
             'm': 2,
             'c': 2,
             'n': 3,
             'q': 2,
             'a': 4,
             'h': 2,
             't': 1,
             'v': 1,
             'r': 1,
             'f': 1,
             'y': 1,
             'g': 1})

In [14]:
# g
zero = (0, 0)

def seqOp(acc, w):
    return (acc[0] + 1, acc[1] + len(w))

def combOp(a, b):
    return (a[0] + b[0], a[1] + b[1])

n_palabras, suma_caracteres = rdd_texto.aggregate(zero, seqOp, combOp)
lonxitude_media = suma_caracteres / n_palabras if n_palabras else 0

(n_palabras, suma_caracteres, lonxitude_media)

(33, 140, 4.242424242424242)

6. A partir de los ficheros /data/notas/notas_mates.txt, /data/notas/notas_fisica.txt y /data/notas/notas_ingles.txt, realiza las siguientes operaciones:
   - a) Crea 3 RDD de pares uno para cada asignatura (notas_mates, notas_ingles, notas_fisica) (0,5)
   - b) Crea un RDD que sea la unión de los 3 anteriores (0,5)
   - c) Muestra la nota más baja de cada alumno (0,5)
   - d) Muestra la nota media de cada alumno (0,5)
   - e) Muestra cuantos estudiantes suspenden cada asignatura (0,5)
   - f) Muestra en qué asignatura suspende más gente (0,5)
   - g) Obtén un RDD que que asocie a cada alumno con las notas en todas las asignaturas (1)

In [2]:
# a)
parse_nota = lambda linea: (linea.split(',')[0], float(linea.split(',')[1]))
notas_mates  = sc.textFile("/data/notas/notas_mates.txt").map(parse_nota)
notas_fisica = sc.textFile("/data/notas/notas_fisica.txt").map(parse_nota)
notas_ingles = sc.textFile("/data/notas/notas_ingles.txt").map(parse_nota)
notas_ingles.take(10)


[('Angel', 4.0),
 ('Maria', 6.0),
 ('Ramon', 8.0),
 ('Jorge', 5.0),
 ('Susana', 2.0),
 ('Anabel', 7.0),
 ('Rocio', 4.0),
 ('Carlos', 8.0),
 ('Triana', 4.0),
 ('Andres', 6.0)]

In [19]:
# b)
notas_todas = notas_mates.union(notas_fisica).union(notas_ingles)
notas_todas.take(10)

[('Angel', 6.0),
 ('Maria', 2.0),
 ('Ramon', 4.5),
 ('Jorge', 10.0),
 ('Susana', 9.0),
 ('Anabel', 8.0),
 ('Pedro', 5.0),
 ('Rocio', 6.0),
 ('Carlos', 4.0),
 ('Triana', 3.0)]

In [23]:
# c)
nota_min_por_alumno = notas_todas.reduceByKey(lambda a, b: a if a < b else b)
nota_min_por_alumno.take(10)

[('Angel', 4.0),
 ('Carlos', 4.0),
 ('Anabel', 2.0),
 ('Jose Juan', 3.0),
 ('Jorge', 5.0),
 ('Susana', 2.0),
 ('Andres', 4.0),
 ('Fernando', 5.0),
 ('Oscar', 3.0),
 ('Isabel', 7.0)]

In [28]:
# d)

sum_count = notas_todas.combineByKey(
    lambda nota: (nota, 1),                 # createCombiner
    lambda acc, nota: (acc[0] + nota, acc[1] + 1),  # mergeValue
    lambda a, b: (a[0] + b[0], a[1] + b[1])         # mergeCombiners
)
nota_media_por_alumno = sum_count.mapValues(lambda sc_: sc_[0] / sc_[1] if sc_[1] else 0.0)

nota_media_por_alumno.take(10)

[('Angel', 6.333333333333333),
 ('Carlos', 5.333333333333333),
 ('Anabel', 5.666666666666667),
 ('Jose Juan', 3.6666666666666665),
 ('Andres', 4.666666666666667),
 ('Jorge', 6.666666666666667),
 ('Susana', 6.666666666666667),
 ('Rocio', 5.5),
 ('Fernando', 7.0),
 ('Oscar', 5.0)]

In [3]:
# e)
suspenden_mates = notas_mates.filter(lambda x: x[1] < 5).count()
suspenden_fisica = notas_fisica.filter(lambda x: x[1] < 5).count()
suspenden_ingles = notas_ingles.filter(lambda x: x[1] < 5).count()

suspensos_por_asignatura = sc.parallelize([("mates",suspenden_mates),("fisica",suspenden_fisica),("ingles",suspenden_ingles)])


print(suspensos_por_asignatura.collect())

Estudiantes que suspenden Matemáticas: 7
Estudiantes que suspenden Física: 8
Estudiantes que suspenden Inglés: 7
[('mates', 7), ('fisica', 8), ('ingles', 7)]


In [5]:
# f)

suspensos_por_asignatura.max()

('mates', 7)

In [32]:
# g)
rdd_total = notas_mates.join(notas_fisica).join(notas_ingles).mapValues(lambda t: (t[0][0], t[0][1], t[1]))
rdd_total.collect()

[('Carlos', (4.0, 4.0, 8.0)),
 ('Angel', (6.0, 9.0, 4.0)),
 ('Anabel', (8.0, 2.0, 7.0)),
 ('Jose Juan', (5.0, 3.0, 3.0)),
 ('Jorge', (10.0, 5.0, 5.0)),
 ('Susana', (9.0, 9.0, 2.0)),
 ('Andres', (4.0, 4.0, 6.0)),
 ('Rocio', (6.0, 5.0, 4.0)),
 ('Rocio', (6.0, 7.0, 4.0)),
 ('Fernando', (5.0, 9.0, 7.0)),
 ('Isabel', (8.0, 8.0, 7.0)),
 ('Alejandro', (5.0, 3.0, 7.0)),
 ('Oscar', (7.0, 5.0, 3.0)),
 ('Leonardo', (1.0, 6.0, 4.0)),
 ('Nicolas', (2.0, 7.0, 5.0)),
 ('Ramon', (4.5, 7.0, 8.0)),
 ('Maria', (2.0, 3.0, 6.0)),
 ('Triana', (3.0, 3.0, 4.0)),
 ('Rosa', (6.0, 8.0, 9.0))]